# Mean Variance Optimization with Scipy

In [3]:
import sys
sys.path.insert(0,'C:\\code\\python_for_the_financial_economist\\')

# import relevant packages
import numpy as np
import pandas as pd
from scipy import stats, optimize
import matplotlib.pyplot as plt

# own functions from codelib
from codelib.statistics import moments as mom
from codelib.portfolio_optimization.mean_variance import portfolio_mean, portfolio_std, portfolio_variance, minimum_variance_portfolio

from codelib.visualization.layout import DefaultStyle
DefaultStyle();

Consider the portfolio optimization problem 


where the expected return vector is given by 

$$
\boldsymbol{\mu} = \begin{pmatrix} 0.032 \\ 0.0322 \\ 0.084 \\ 0.082 \end{pmatrix},
$$

the vector of volatilites is given by 

$$
\mathbf{v} = \begin{pmatrix} 0.05 \\ 0.05 \\ 0.22 \\ 0.22 \end{pmatrix},
$$

and the correlation matrix is given by 

$$
\mathbf{C} = \begin{pmatrix} 1.0 & 0.85 & 0.5 & 0.45 \\
                      0.85 & 1.0 & 0.5 & 0.45 \\
                      0.5 & 0.5 & 1.0 & 0.9 \\
                      0.45 & 0.45 & 0.9 & 1.0 \end{pmatrix}
$$

In [2]:
corr_mat = np.array([[1.0, 0.85, 0.5, 0.45],
                     [0.85, 1.0, 0.5, 0.45],
                     [0.5, 0.5, 1.0, 0.9],
                     [0.45, 0.45, 0.9, 1.0]])

vols = np.array([5.0, 5.0, 22.0, 22.0]) / 100.0
mu = np.array([3.2, 3.22, 8.4, 8.2]) / 100.0

cov_mat = mom.corr_to_cov_matrix(corr_mat, vols)

## Problem 1

Set up the minimization problem that enable you to find the minimum variance portfolio using `scipy.optimize`. Apply a budget constraint requiring the portfolio weights to sum to one. Compare with the analytical solution. 

### Solution

We have the minimization problem 

$$
\mathbf{w}^{Min-Var} = \arg \min \mathbf{w}^\top \boldsymbol{\Sigma} \mathbf{w} \; \; \text{st. } \mathbf{1}^\top \mathbf{w} = 1
$$

The analytical solution is given by 

$$
\mathbf{w}^{Min-Var} = \frac{\boldsymbol{\Sigma}^{-1} \mathbf{1}}{\mathbf{1}^\top \boldsymbol{\Sigma}^{-1} \mathbf{1}}
$$

In [5]:
"""
Solving with scipy
"""

# define sum to one constraint
sum_to_one_cons = {'type': 'eq',
                   'fun' : lambda x: np.sum(x) - 1.0, 
                   'jac' : lambda x: np.ones_like(x)} 

# define initial guess 
x0 = np.ones_like(mu) / len(mu)

# define Jacobian 
port_var_der = lambda w, cov_mat: 2 * w @ cov_mat


res = optimize.minimize(portfolio_variance, x0=x0, args=(cov_mat,),
                        method='SLSQP',
                        jac=port_var_der,
                        constraints=[sum_to_one_cons],  # no_short_cons,
                        options={'ftol': 1e-9, 'disp': True}) #, bounds=bounds)

w_min_var_opt = res.x

# show results
res

Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.002052067836953288
            Iterations: 8
            Function evaluations: 8
            Gradient evaluations: 8


 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 0.002052067836953288
       x: [ 5.389e-01  5.389e-01 -1.002e-01  2.231e-02]
     nit: 8
     jac: [ 4.104e-03  4.104e-03  4.104e-03  4.104e-03]
    nfev: 8
    njev: 8

In [8]:
"""
Solving with scipy - not providing gradient information
"""

# define sum to one constraint
sum_to_one_cons = {'type': 'eq',
                   'fun' : lambda x: np.sum(x) - 1.0} #<- this part is unnecessary

# define initial guess 
x0 = np.ones_like(mu) / len(mu)

res = optimize.minimize(portfolio_variance, x0=x0, args=(cov_mat,),
                        method='SLSQP',
                        constraints=[sum_to_one_cons],  # no_short_cons,
                        options={'ftol': 1e-9, 'disp': True}) #, bounds=bounds)

w_min_var_opt = res.x

# show results
res

Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.0020520678369532913
            Iterations: 8
            Function evaluations: 40
            Gradient evaluations: 8


 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 0.0020520678369532913
       x: [ 5.389e-01  5.389e-01 -1.002e-01  2.231e-02]
     nit: 8
     jac: [ 4.104e-03  4.104e-03  4.104e-03  4.104e-03]
    nfev: 40
    njev: 8

In [11]:
w_min_var_opt

array([ 0.53892691,  0.53892691, -0.10016861,  0.02231479])

In [9]:
minimum_variance_portfolio(cov_mat)

array([ 0.53892691,  0.53892691, -0.1001686 ,  0.02231479])

## Problem 2

Minimize the portfolio variance subject to the constraints that 

* The expected return should be above 6%
* No shorting (all weights should be equal or greater than 0)
* The portfolio weights need to sum to one (budget constraint)

### Solution

In [18]:
# budget constraint 
sum_to_one_cons = {'type': 'eq',
                   'fun' : lambda x: np.sum(x) - 1.0, 
                   'jac' : lambda x: np.ones_like(x)}

# not shorting constraint
no_short_cons = {'type': 'ineq',
                 'fun' : lambda x: x,
                 'jac' : lambda x: np.eye(len(x))}

# alternatively use 
bounds = [(0.0, 1.0)] * len(mu)

# define return constraint 
target_cons = {'type': 'ineq',
               'fun' : lambda x: x @ mu - 0.06,
               'jac' : lambda x: mu}

port_var_der = lambda w, cov_mat: 2 * w @ cov_mat


x0 = np.ones_like(mu) / len(mu)
res = optimize.minimize(portfolio_variance, x0=x0, args=(cov_mat,),
                        method='SLSQP',
                        jac=port_var_der,
                        constraints=[sum_to_one_cons, no_short_cons,  target_cons],  # no_short_cons,
                        options={'ftol': 1e-9, 'disp': True}) #, bounds=bounds)

w_mean_var_opt = res.x

res

Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.016816396516555813
            Iterations: 12
            Function evaluations: 12
            Gradient evaluations: 12


 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 0.016816396516555813
       x: [ 1.044e-01  3.506e-01  3.420e-01  2.029e-01]
     nit: 12
     jac: [ 7.783e-03  7.968e-03  5.579e-02  5.394e-02]
    nfev: 12
    njev: 12

In [19]:
w_mean_var_opt

array([0.10444791, 0.35063358, 0.34197387, 0.20294464])